In [2]:
import numpy as np
import pandas as pd

import keras
from keras.preprocessing import image

from tqdm import tqdm

Using TensorFlow backend.


In [3]:
movies = pd.read_csv('archive/movies_metadata.csv')
print("Shape: ", movies.shape)
print("Features: ", movies.columns)

C:\Python\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Shape:  (45466, 24)
Features:  Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [19]:
movies.head()['imdb_id']

0    tt0114709
1    tt0113497
2    tt0113228
3    tt0114885
4    tt0113041
Name: imdb_id, dtype: object

In [5]:
movies.head()['genres']

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [6]:
movies.head()['poster_path']

0    /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1    /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2    /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3    /16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4    /e64sOI48hQXyru7naBFyssKFxVd.jpg
Name: poster_path, dtype: object

In [20]:
movies.head()['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [16]:
# ARCHIVED FUNCTION
"""
def f(downloaded_image_dir, title, poster_path):
    if not os.path.exists(downloaded_image_dir):
        os.makedirs(downloaded_image_dir)
    
    # corrupted url
    imgUrl = "https://www.themoviedb.org/t/p/w1280" + str(poster_path)
    filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'

    try:
        r = requests.get(imgUrl, stream=True)
        if r.status_code == 200:
            with open(downloaded_image_dir + '/' + filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            print("Not Found: ", title, imgUrl)
        
    except:
        print("Error Dowloading: ", title, imgUrl)    
"""

Not Found:  Toy Story https://www.themoviedb.org/t/p/w1280/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
Not Found:  Grumpier Old Men https://www.themoviedb.org/t/p/w1280/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
Not Found:  Waiting to Exhale https://www.themoviedb.org/t/p/w1280/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
Not Found:  Father of the Bride Part II https://www.themoviedb.org/t/p/w1280/e64sOI48hQXyru7naBFyssKFxVd.jpg
Not Found:  Heat https://www.themoviedb.org/t/p/w1280/zMyfPUelumio3tiDKPffaUpsQTD.jpg
Not Found:  Sabrina https://www.themoviedb.org/t/p/w1280/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg
Not Found:  Tom and Huck https://www.themoviedb.org/t/p/w1280/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg
Not Found:  Sudden Death https://www.themoviedb.org/t/p/w1280/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg
Not Found:  GoldenEye https://www.themoviedb.org/t/p/w1280/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg
Not Found:  The American President https://www.themoviedb.org/t/p/w1280/lymPNGLZgPHuqM29rKMGV46ANij.jpg
Not Found:  Dracula: Dead and Loving It https

In [45]:
# @Source: https://johannesbader.ch/blog/tutorial-download-posters-with-the-movie-database-api-in-python/"

import os
import re
import requests

CONFIG_PATTERN = 'http://api.themoviedb.org/3/configuration?api_key={key}'
IMG_PATTERN = 'http://api.themoviedb.org/3/movie/{imdbid}/images?api_key={key}' 
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()
    
def _download_images(titles, urls, path):
    """download all images in list 'urls' to 'path' """
    print(len(urls), len(titles))
    zipped = list(zip(titles, urls))
    for title, url in zipped:
        r = requests.get(url)
        filetype = r.headers['content-type'].split('/')[-1]
        filename = re.sub(r'\W+', ' ', title).lower().strip().replace(" ", "-") + '.jpg'
        filepath = os.path.join(path, filename)
        
        if r.status_code == 200:
            with open(filepath,'wb') as w:
                w.write(r.content)

def get_poster_url(imdbid):
    """ 
    given IMDB id, returns the corresponding url for
    poster jpg
    
    Args: 
        imdbid (str): IMDB id
    Return: 
        url (str): image url of poster image from 'themoviedb.org'
                of the maximum available size. 
    """
    config = _get_json(CONFIG_PATTERN.format(key=KEY))
    base_url = config['images']['base_url']
    sizes = config['images']['poster_sizes']   # sorted in ascending order
    
    def size_str_to_int(x):
        return float("inf") if x == 'original' else int(x[1:])
    
    # retrieve image of max size
    max_size = max(sizes, key=size_str_to_int)
    
    posters = _get_json(IMG_PATTERN.format(key=KEY,imdbid=imdbid))['posters']
    
    if len(posters) == 0:
        return None
    
    rel_path = posters[0]['file_path']
    url = "{0}{1}{2}".format(base_url, max_size, rel_path)
    
    return url

In [46]:
# RUN THIS FUNCTION TO DOWNLOAD JPG IMAGES
def tmdb_posters():
    """ download posters with imdb_id from dataset """
    
    if not os.path.exists('images_movies_genre'):
        os.makedirs('images_movies_genre')
    
    titles = []
    urls = []
    
    n = 0
    for index, row in movies.iterrows():
        title = str(row['title'])
        url = get_poster_url(row['imdb_id'])
        
        # skip iteration if poster cannot be located
        if not url:
            continue
            
        titles.append(title)
        urls.append(url)
        n += 1
        
        # SET LIMIT FOR NUMBER OF IMAGES TO DOWNLOAD
        if n > 20:
            break

    _download_images(titles, urls, 'images_movies_genre')

tmdb_posters() 

22 22


In [ ]:
# TODO: jpg --> numpy array conversion

X = []

for i in tqdm(range(movies.shape[0])):
    img = image.load_img(path, target_size=(350, 350, 3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    
X = np.array(X)

In [ ]:
# TODO: Padding
# reference here: https://jinglescode.github.io/2019/11/17/predict-movie-earnings-with-posters/

# Create a larger zero canvas
canvas = np.zeros(512, 512, 3)

# Overlay with image
canvas[6:506, 6:506] = your_500_500_img